In [38]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

import glob
import cv2
from PIL import Image

import os
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

import random

In [39]:
np.random.seed(1000)

In [40]:
# Get Data
filelist_B5 = [cv2.imread(file) for file in glob.glob("dataset/*B5*.jpg")]
x_B5 = np.array([np.array(Image.fromarray(fname)) for fname in filelist_B5])
y_B5 = np.zeros((x_B5.shape[0],1))

filelist_A4 = [cv2.imread(file) for file in glob.glob("dataset/*A4*.jpg")]
x_A4 = np.array([np.array(Image.fromarray(fname)) for fname in filelist_A4])
y_A4 = np.ones((x_A4.shape[0],1))

x = np.concatenate((x_B5, x_A4))
y = np.concatenate((y_B5, y_A4))


In [41]:
test_idx = random.sample(range(0, x.shape[0]), int(x.shape[0]*0.1))
x_test = x[test_idx]
x_train = np.delete(x, test_idx, 0)
y_test = y[test_idx]
y_train = np.delete(y, test_idx, 0)

y_train = to_categorical(y_train)

In [42]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(500,500,3), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(500*500*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 123, 123, 96)      34944     
_________________________________________________________________
activation_10 (Activation)   (None, 123, 123, 96)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 61, 96)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 61, 61, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 51, 51, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 51, 51, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 256)       0         
__________

In [43]:
# Compile 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [53]:
# Train
model.fit(x_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.1, shuffle=True)

Train on 62 samples, validate on 7 samples
Epoch 1/50
62/62 [==============================] - 125s 2s/step - loss: 0.5692 - acc: 0.7581 - val_loss: 6.4925 - val_acc: 0.0000e+00
Epoch 2/50
62/62 [==============================] - 114s 2s/step - loss: 0.3559 - acc: 0.8387 - val_loss: 7.7960 - val_acc: 0.1429
Epoch 3/50
62/62 [==============================] - 123s 2s/step - loss: 0.4176 - acc: 0.8387 - val_loss: 6.8633 - val_acc: 0.0000e+00
Epoch 4/50
62/62 [==============================] - 144s 2s/step - loss: 0.4217 - acc: 0.8387 - val_loss: 6.3779 - val_acc: 0.2857
Epoch 5/50
62/62 [==============================] - 142s 2s/step - loss: 0.7535 - acc: 0.7581 - val_loss: 5.1251 - val_acc: 0.2857
Epoch 6/50
62/62 [==============================] - 136s 2s/step - loss: 0.5848 - acc: 0.8065 - val_loss: 3.3873 - val_acc: 0.4286
Epoch 7/50
62/62 [==============================] - 140s 2s/step - loss: 0.5594 - acc: 0.7903 - val_loss: 6.8821 - val_acc: 0.1429
Epoch 8/50
62/62 [==============

In [54]:
# Test
y_pred = model.predict_classes(x_test)
cmat = confusion_matrix(y_test, y_pred)

In [55]:
cmat

array([[3, 1],
       [1, 2]], dtype=int64)